## SLIDERULE DOWNLOADER FOR ATL03 DATA

In [1]:
#IMPORTS

import pyarrow.feather as feather

import socket

import sys
# import custom modules
if socket.gethostname() == 'ra':
    basepath = '/home/marcomaz/snowdepth/'
else :    
    basepath = '/uio/hypatia/geofag-felles/projects/snowdepth/'
mytoolspath = basepath + 'code/tools/own_repos/snowdepth/tools'
sys.path.append(mytoolspath)
#import IC2tools
import numpy as np


import sliderule
from sliderule import icesat2, ipysliderule, io, earthdata
import ipywidgets as widgets
import logging
import warnings
# autoreload
%load_ext autoreload
%autoreload 2
# turn off warnings for demo
warnings.filterwarnings('ignore')



In [7]:
import geopandas as gpd

In [10]:
east_river = gpd.read_file('./shp/eastriver.shp')

In [13]:
coords = east_river.geometry.total_bounds

In [3]:
# general settings and functions
wgs84 = 'EPSG:4326'
def coords2poly(coords):
    poly=[{'lon': coords[0], 'lat': coords[1]},
     {'lon': coords[2], 'lat': coords[1]},
     {'lon':coords[2], 'lat': coords[3]},
     {'lon': coords[0], 'lat': coords[3]},
     {'lon': coords[0], 'lat': coords[1]}]
    return poly

asset = 'icesat2'
release = '006'


Initialize the connection to Icesat2

In [2]:
icesat2.init(url=sliderule.service_url, verbose=False)

Define the region of interest.

In [16]:
wgs84 = 'EPSG:4326'
utm30n = 'EPSG:25830'

#coords_S = [-0.53, 42.659, -0.17, 42.84]

region=coords2poly(coords)

Define the parameters for the download

#old parameters settings for Izas
resources = icesat2.cmr(short_name='ATL03', polygon=region, time_start='2022-06-01',
    time_end='2022-07-20')#, asset=asset) 

parms = {
    "time_start":'2021-01-01',
    "time_end":'2021-05-01',
    "poly": region,
    #"srt": icesat2.SRT_LAND,
    #minimum confidence level
    "cnf": 1,
    "quality_ph":0,
    
    #uncomment next line if you want only ground classified photons
    #"atl08_class":"atl08_ground",
    
    """
    "yapc":{#filter parameters
            "score":0,
            "knn": 0,
            "min_ph":0,
            #parameters for the yapc score window (where to look for neighboors)
            "win_h":3,
            "win_x":5
            },"""
    #filter parameters
    "cnt": 3,
    #definition of a segment
    "len": 5.0,
    "res": 5.0,
    "ats":1,
}


In [18]:
asset = 'icesat2'

time_start='2022-10-01'
time_end  ='2023-10-01'

parms = {
    # processing parameters
    "srt": icesat2.SRT_LAND,
    "len": 20,
    # classification and checks
    # still return photon segments that fail checks
    "pass_invalid": True, 
    # all photons
    "cnf": -2, 
    # all land classification flags
    "atl08_class": ["atl08_noise", "atl08_ground", "atl08_canopy", "atl08_top_of_canopy", "atl08_unclassified"],    # all photons
    "yapc": dict(knn=0, win_h=3, win_x=5, min_ph=0, score=0), 
}
release = '006'

granules_list = earthdata.cmr(short_name='ATL03', polygon=region, time_start=time_start, time_end=time_end, version=release)

parms["poly"] = region

Check the dates of available data

In [19]:
granules_list

['ATL03_20221002012219_01731702_006_01.h5',
 'ATL03_20221029121409_05921706_006_01.h5',
 'ATL03_20221202222559_11181702_006_02.h5',
 'ATL03_20221230091748_01501806_006_02.h5',
 'ATL03_20221231210202_01731802_006_02.h5',
 'ATL03_20230128075403_05921806_006_02.h5',
 'ATL03_20230303180553_11181802_006_02.h5',
 'ATL03_20230331045749_01501906_006_02.h5',
 'ATL03_20230401164153_01731902_006_02.h5',
 'ATL03_20230429033339_05921906_006_02.h5',
 'ATL03_20230602134520_11181902_006_02.h5',
 'ATL03_20230630003657_01502006_006_02.h5',
 'ATL03_20230701122109_01732002_006_02.h5',
 'ATL03_20230728231251_05922006_006_02.h5',
 'ATL03_20230901092431_11182002_006_02.h5',
 'ATL03_20230928201621_01502106_006_02.h5',
 'ATL03_20230930080030_01732102_006_02.h5']

Execute the request

In [55]:
atl03 = icesat2.atl03sp(parms, asset=asset, version=release, resources=granules_list)

here is the data in geoDataFrame

In [56]:
atl03

,cycle,background_rate,rgt,segment_id,sc_orient,track,segment_dist,solar_elevation,quality_ph,yapc_score,relief,height,landcover,snowcover,distance,atl08_class,atl03_cnf,pair,geometry,spot
time,,,,,,,,,,,,,,,,,,,,
2020-02-05 04:51:36.848847104,6,5493.807615,618,764048,1,1,1.530352e+07,-26.149235,0,0,0.0,1956.522461,255,255,-9.794150,0,4,0,POINT (-0.41705 42.84008),6
2020-02-05 04:51:36.848847104,6,5493.807615,618,764048,1,1,1.530352e+07,-26.149235,0,52,0.0,1944.862183,255,255,-9.761000,3,4,0,POINT (-0.41705 42.84008),6
2020-02-05 04:51:36.848847104,6,5493.807615,618,764048,1,1,1.530352e+07,-26.149235,0,0,0.0,1954.428833,255,255,-9.788300,0,4,0,POINT (-0.41705 42.84008),6
2020-02-05 04:51:36.848847104,6,5493.807615,618,764048,1,1,1.530352e+07,-26.149235,0,169,0.0,1943.989868,255,255,-9.758562,3,4,0,POINT (-0.41705 42.84008),6
2020-02-05 04:51:36.848847104,6,5493.807615,618,764048,1,1,1.530352e+07,-26.149235,0,222,0.0,1943.155396,255,255,-9.756368,2,4,0,POINT (-0.41705 42.84008),6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-05 04:51:40.083347200,6,6433.464472,618,765086,1,3,1.532432e+07,-26.221310,0,9,0.0,1105.201538,255,255,5.300246,0,4,1,POINT (-0.52028 42.65909),1
2020-02-05 04:51:40.083647232,6,6433.464472,618,765086,1,3,1.532432e+07,-26.221310,0,0,0.0,1111.386841,255,255,7.443409,2,4,1,POINT (-0.52028 42.65907),1
2020-02-05 04:51:40.083647232,6,6433.464472,618,765086,1,3,1.532432e+07,-26.221310,0,24,0.0,1119.040894,255,255,7.458418,3,4,1,POINT (-0.52027 42.65907),1


Handle the downloaded data geometry and save it as a feather.

In [57]:
atl03= atl03.to_crs(epsg='25830')
atl03['x'] = atl03.geometry.x
atl03['y'] = atl03.geometry.y

atl03 = (atl03.drop(columns={'geometry'}))

with open('ATL03_yapc.feather', 'wb') as f:
    feather.write_feather(atl03, f)

Next is code for filtering the result or check the distribution of some attributes

In [16]:
atl03[atl03.index>'2019-01-01']

,segment_id,segment_dist,cycle,rgt,sc_orient,track,quality_ph,height,atl03_cnf,atl08_class,delta_time,distance,yapc_score,pair,spot,x,y
time,,,,,,,,,,,,,,,,,
2019-02-04 10:04:53.948765984,236739,4.747472e+06,2,580,0,3,0,1151.406982,2,4,3.450989e+07,1.168355,0,0,5,720770.570066,4.726478e+06
2019-02-04 10:04:53.948765984,236739,4.747472e+06,2,580,0,3,0,1141.324463,3,4,3.450989e+07,1.228115,0,0,5,720770.674218,4.726478e+06
2019-02-04 10:04:53.948765984,236739,4.747472e+06,2,580,0,3,0,1149.252319,2,4,3.450989e+07,1.181160,0,0,5,720770.592324,4.726478e+06
2019-02-04 10:04:53.948865980,236739,4.747477e+06,2,580,0,3,0,1142.261108,3,4,3.450989e+07,-3.061166,69,0,5,720770.575168,4.726479e+06
2019-02-04 10:04:53.948865980,236739,4.747477e+06,2,580,0,3,0,1140.234985,3,4,3.450989e+07,-3.049338,42,0,5,720770.596099,4.726479e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01 09:30:21.150827072,765006,1.532270e+07,16,618,1,1,0,1079.160522,2,4,1.445814e+08,-3.311523,20,1,5,720295.815529,4.726457e+06
2022-08-01 09:30:21.150927072,765006,1.532270e+07,16,618,1,1,0,1078.251465,2,4,1.445814e+08,-2.607821,56,1,5,720295.791562,4.726457e+06
2022-08-01 09:30:21.151027072,765006,1.532270e+07,16,618,1,1,0,1080.808838,2,4,1.445814e+08,-1.890160,0,1,5,720295.703448,4.726456e+06


In [ ]:
#geometrical filtering
goodpoints = (atl03.geometry.x > coords_S[0])&(atl03.geometry.x < coords_S[2])&(atl03.geometry.y > coords_S[1])&(atl03.geometry.y < coords_S[3])


In [27]:
#atl03[goodpoints].to_csv('ATL03_ground_yapc.csv')

In [ ]:
# !! track is defined this way!!
atl03.track.unique()